In [50]:
import sys, os, re
import boto3, urllib3

import pandas as pd

from GOSTrocks.misc import tPrint 

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty"
region = "us-east-1"
s3client = boto3.client("s3", region_name=region, verify=False)

# Loop through the S3 bucket and get all the file keys
more_results = True
try:
    del token  # noqa
except Exception:
    pass
loops = 0

all_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket,
            ContinuationToken=token,  # noqa
            Prefix=prefix,  # noqa
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        # Check if the key ends with .csv
        if res["Key"].endswith("_1k.txt"):
            all_res.append(res)

Completed loop: 0
Completed loop: 1
Completed loop: 2
Completed loop: 3
Completed loop: 4
Completed loop: 5
Completed loop: 6
Completed loop: 7
Completed loop: 8
Completed loop: 9
Completed loop: 10
Completed loop: 11
Completed loop: 12
Completed loop: 13
Completed loop: 14
Completed loop: 15
Completed loop: 16
Completed loop: 17
Completed loop: 18
Completed loop: 19
Completed loop: 20
Completed loop: 21


'AGO'

In [54]:
def extract_numbers(text):
    res = re.findall(r'\d+', text)
    if len(res) == 0:
        return None
    else:
        return float(res[0])

country_population_res = []
for res in all_res:
    # Get text file from S3
    res_text = s3client.get_object(Bucket=bucket, Key=res["Key"])['Body'].read()
    cntry = os.path.basename(res['Key']).split("_")[-2]
    tPrint(f"Processing {cntry}...")
    # Go through lines of results and extract urbanization numbers
    res_lines = []
    for line in res_text.decode('utf-8').split('\n'):
        if line == "***** Evaluate Urbanization *****":
            record_results = True
        elif line == "***** Evaluate SMOD ******":
            record_results = False
        if record_results:
            res_lines.append(line)

    # Extract the summary results for all population layers
    res_lines = res_lines[1:]  # Remove the first line which is a header
    all_pop_res = []
    for line in res_lines:
        line_split = line.split(":")    
        pop_layer = line_split[0].strip()
        pop_res = [extract_numbers(x) for x in line_split[1:]]    
        pop_res.append(pop_layer)
        if len(pop_res) >= 4:
            all_pop_res.append(pop_res[1:])
    country_res = pd.DataFrame(all_pop_res, columns=["Population", "HD_Urban", "Urban", "PopLayer"])
    country_res["Country"] = cntry
    country_population_res.append(country_res)


09:59:18	Processing AGO...
09:59:18	Processing BDI...
09:59:18	Processing BEN...
09:59:18	Processing BFA...
09:59:18	Processing BGD...
09:59:19	Processing BWA...
09:59:19	Processing CAF...
09:59:19	Processing CIV...
09:59:19	Processing CMR...
09:59:19	Processing COD...
09:59:19	Processing COG...
09:59:19	Processing COL...
09:59:19	Processing COM...
09:59:19	Processing CPV...
09:59:19	Processing EGY...
09:59:19	Processing ERI...
09:59:19	Processing ETH...
09:59:20	Processing GAB...
09:59:20	Processing GHA...
09:59:20	Processing GIN...
09:59:20	Processing GMB...
09:59:20	Processing GNB...
09:59:20	Processing KEN...
09:59:20	Processing LBR...
09:59:20	Processing LSO...
09:59:20	Processing MDG...
09:59:20	Processing MLI...
09:59:20	Processing MOZ...
09:59:21	Processing MRT...
09:59:21	Processing MUS...
09:59:21	Processing MWI...
09:59:21	Processing NAM...
09:59:21	Processing NER...
09:59:21	Processing NGA...
09:59:21	Processing RWA...
09:59:21	Processing SDN...
09:59:21	Processing SEN...
0

In [55]:
pd.concat(country_population_res).to_csv("C:/WBG/Work/temp/urbanization_country_population.csv", index=False)

In [30]:
res_lines[1].split(":")

['WP_U_15',
 ' TotalPop',
 ' 25693931.0, UrbanPop',
 ' 11920089.0, HD Pop',
 ' 8586900.0']

In [33]:
res_lines

['***** Evaluate Urbanization *****',
 'WP_U_15: TotalPop: 25693931.0, UrbanPop: 11920089.0, HD Pop: 8586900.0',
 'WP_U_15: 46.39% Urban; 33.42% HD Urban',
 'WP_C_15: TotalPop: 25608675.0, UrbanPop: 15496020.0, HD Pop: 12333667.0',
 'WP_C_15: 60.51% Urban; 48.16% HD Urban',
 'WP_C_20: TotalPop: 36374021.0, UrbanPop: 25936012.0, HD Pop: 22636215.0',
 'WP_C_20: 71.3% Urban; 62.23% HD Urban',
 'GHS: TotalPop: 25058900.0, UrbanPop: 21058336.0, HD Pop: 16906967.0',
 'GHS: 84.04% Urban; 67.47% HD Urban',
 'GHS: TotalPop: 25058900.0, UrbanPop: 21058336.0, HD Pop: 16906967.0',
 'GHS: 84.04% Urban; 67.47% HD Urban']

In [ ]:
_l